In [1]:
using LinearAlgebra
function givensrot(A, i, j)
  q = sqrt(A[j,j]^2 + A[i,j]^2);
  c = A[j,j] / q;
  s = A[i,j] / q;
  N = size(A, 1);
  G = diagm(ones(N));
  G[i,i] = c; G[j,j] = c;
  G[j,i] = s; G[i,j] =-s;
  return G
end

givensrot (generic function with 1 method)

In [2]:
A = rand(3,3)

3×3 Matrix{Float64}:
 0.300087  0.141856  0.63877
 0.557163  0.82042   0.889409
 0.362941  0.253819  0.693175

In [3]:
A1 = copy(A)
G1 = givensrot(A1, 2, 1)
A2 = G1 * A1

3×3 Matrix{Float64}:
 0.632837  0.789583   1.08595
 0.0       0.264144  -0.140636
 0.362941  0.253819   0.693175

In [4]:
G2 = givensrot(A2, 3, 1)
A3 = G2 * A2

3×3 Matrix{Float64}:
 0.729527   0.811209   1.28688
 0.0        0.264144  -0.140636
 0.0       -0.172641   0.0610387

In [5]:
G3 = givensrot(A3, 3, 2)
A4 = G3 * A3

3×3 Matrix{Float64}:
 0.729527  0.811209   1.28688
 0.0       0.315558  -0.151116
 0.0       0.0       -0.0258478

In [8]:
Q = (G1' * G2' * G3')
Q' * Q

3×3 Matrix{Float64}:
 1.0           0.0           5.55112e-17
 0.0           1.0          -5.55112e-17
 5.55112e-17  -5.55112e-17   1.0

In [9]:
R = (G3 * (G2 * (G1 * A)))

3×3 Matrix{Float64}:
 0.729527  0.811209   1.28688
 0.0       0.315558  -0.151116
 0.0       0.0       -0.0258478

In [10]:
norm(Q * R - A)

1.1443916996305594e-16

In [16]:
using SparseArrays
dropalmostzeros(A, ϵ) = sparse([abs(aij) > ϵ ? aij : 0.0 for aij in A])

function householder_hessenberg(A :: Matrix{Float64})
  H = copy(A)
  N = size(A, 1)
  for i=1:N-2
    α = H[i+1:N, i] # FIXME
    σ = norm(α)
    β = 1.0  / (σ * (σ + abs(α[1])))
    expjϕ = α[1] < 0.0 ? -1.0 : 1.0 
    u = copy(α)
    u[1] = expjϕ * (σ + abs(α[1]))
    T = diagm(ones(N))
    T[i+1:N, i+1:N] .= diagm(ones(length(α))) - β * u * u'
    H .= T * H * T
  end
  return H
end

function francis(A :: Matrix{Float64}, maxiter)
  H = householder_hessenberg(A)
  N = size(H, 1)
  G = [[0. 0.; 0. 0.] for j=1:N-1]
  λs = zeros(N)
  for k = 1:maxiter
    for j = 1:N-1
      q = sqrt(H[j,j]^2 + H[j+1,j]^2);
      c = H[j,  j] / q;
      s = H[j+1,j] / q;
      G[j] = [c -s; +s c]
      H[j:j+1, j:N] = G[j]' * H[j:j+1, j:N] # => 2 x (N-j)
    end
    for j = 1:N-1
      H[1:j+1, j:j+1] = H[1:j+1, j:j+1] * G[j] # (j) x 2
    end
    λs .= diag(H)
  end
display(dropalmostzeros(H, 1e-15))
  return λs
end

francis (generic function with 1 method)

In [22]:
A = rand(5, 5)
francis(A, 1000)

5×5 SparseMatrixCSC{Float64, Int64} with 17 stored entries:
 2.32667  -0.480857  -0.0262013  -0.0754038    -0.314329
  ⋅        0.521735   0.199474    0.0789113    -0.44059
  ⋅         ⋅         0.468536   -0.454811      0.167329
  ⋅         ⋅         0.468455   -0.00935293    0.320666
  ⋅         ⋅          ⋅          1.53687e-15  -0.442669

5-element Vector{Float64}:
  2.326671065695723
  0.5217345454060954
  0.4685361720453676
 -0.009352934513827845
 -0.44266948019927466

In [24]:
eigvals(A)

5-element Vector{ComplexF64}:
 -0.44266948019927754 + 0.0im
  0.22959161876577067 - 0.39492260618921127im
  0.22959161876577067 + 0.39492260618921127im
   0.5217345454060951 + 0.0im
   2.3266710656957237 + 0.0im

In [25]:
eigvals([0.468536   -0.454811; 0.468455   -0.00935293])

2-element Vector{ComplexF64}:
 0.22959153499999996 - 0.39492281480165686im
 0.22959153499999996 + 0.39492281480165686im

In [28]:
function householder_qr(A :: Matrix{Float64})
  R = copy(A)
  N = size(A, 1)
  Q = diagm(ones(N))
  for i=1:size(A, 2)
    α = R[i:N, i]
    σ = norm(α)
    β = 1.0  / (σ * (σ + abs(α[1])))
    expjϕ = α[1] < 0.0 ? -1.0 : 1.0 
    u = copy(α)
    u[1] = expjϕ * (σ + abs(α[1]))
    T = diagm(ones(N))
    T[i:N, i:N] .= diagm(ones(length(α))) - β * u * u'
    R = T * R # te linie zostały zmienione
    Q = Q * T # względem metody Householdera zdefiniowanej wcześniej
  end
  return Q, R
end

householder_qr (generic function with 1 method)

In [39]:
A = rand(10, 5)
x = rand(5)
b = A * x;

In [40]:
Q, R = householder_qr(A);
dropalmostzeros(R, 1e-15)

10×5 SparseMatrixCSC{Float64, Int64} with 15 stored entries:
 -1.86283  -1.93927  -1.43489   -1.31683    -1.75303
   ⋅        1.07665   0.521982   0.497255    0.324312
   ⋅         ⋅       -1.17397   -0.0965497  -0.131646
   ⋅         ⋅         ⋅        -0.84823     0.337619
   ⋅         ⋅         ⋅          ⋅         -0.610674
   ⋅         ⋅         ⋅          ⋅           ⋅ 
   ⋅         ⋅         ⋅          ⋅           ⋅ 
   ⋅         ⋅         ⋅          ⋅           ⋅ 
   ⋅         ⋅         ⋅          ⋅           ⋅ 
   ⋅         ⋅         ⋅          ⋅           ⋅ 

In [42]:
R1 = R[1:5,  1:5];
Q1 = Q[1:10, 1:5];

In [47]:
x̂ = R1 \ (Q1' * b)

5-element Vector{Float64}:
 0.03261381011816847
 0.7010814888617812
 0.9460530006619281
 0.021745873058307146
 0.13179719391853362

In [48]:
x

5-element Vector{Float64}:
 0.032613810118168884
 0.701081488861781
 0.9460530006619281
 0.021745873058306997
 0.13179719391853362

In [49]:
norm(b - A * x̂)

4.2998752849492583e-16